Copyright (c) 2024 Gabriele Mancari Pasi <s323387@studenti.polito.it>

## Lab 3 - Path search Gem Puzzle

In [1]:
from collections import namedtuple
from random import choice
from tqdm.auto import tqdm
import numpy as np

In [2]:
PUZZLE_DIM = 3
action = namedtuple('Action', ['pos1', 'pos2'])

In [3]:
def available_actions(state: np.ndarray) -> list['Action']:
    x, y = [int(_[0]) for _ in np.where(state == 0)]
    actions = list()
    if x > 0:
        actions.append(action((x, y), (x - 1, y)))
    if x < PUZZLE_DIM - 1:
        actions.append(action((x, y), (x + 1, y)))
    if y > 0:
        actions.append(action((x, y), (x, y - 1)))
    if y < PUZZLE_DIM - 1:
        actions.append(action((x, y), (x, y + 1)))
    return actions

In [4]:
def do_action(state: np.ndarray, action: 'Action') -> np.ndarray:
    new_state = state.copy()
    new_state[action.pos1], new_state[action.pos2] = new_state[action.pos2], new_state[action.pos1]
    return new_state

### Manhattan distance
One of the most common heuristics is the Manhattan distance, a function that estimates the cost to reach the goal from the current state

In [5]:
def manhattan_distance(state: np.ndarray, goal: np.ndarray) -> int:
    """Calculate the Manhattan distance between the current state and the goal"""
    distance = 0
    for x in range(PUZZLE_DIM):
        for y in range(PUZZLE_DIM):
            value = state[x, y]
            if value == 0:
                continue
            target_x, target_y = divmod(value - 1, PUZZLE_DIM)
            distance += abs(x - target_x) + abs(y - target_y)
    return distance

In [6]:
def is_goal(state: np.ndarray, goal: np.ndarray) -> bool:
    """Check if the current state is the goal state"""
    return np.array_equal(state, goal)

### A* approach


In [7]:
def a_star_solve(start_state: np.ndarray, goal_state: np.ndarray) -> list[np.ndarray]:
    """Solve the n^2-1 puzzle using the A* algorithm"""
    frontier = [(0, tuple(start_state.flatten()), start_state)]
    
    came_from = {tuple(start_state.flatten()): None}
    cost_so_far = {tuple(start_state.flatten()): 0}
    
    while frontier:
        _, current_tuple, current_state = frontier.pop(0)
        
        if is_goal(current_state, goal_state):
            path = []
            while current_tuple is not None:
                path.append(current_state)
                current_tuple = came_from[current_tuple]
                current_state = None if current_tuple is None else np.array(current_tuple).reshape(PUZZLE_DIM, PUZZLE_DIM)
            return path[::-1]
        
        for act in available_actions(current_state):
            next_state = do_action(current_state, act)
            next_tuple = tuple(next_state.flatten())
            new_cost = cost_so_far[current_tuple] + 1
            
            if next_tuple not in cost_so_far or new_cost < cost_so_far[next_tuple]:
                cost_so_far[next_tuple] = new_cost
                priority = new_cost + manhattan_distance(next_state, goal_state)

                inserted = False
                for i in range(len(frontier)):
                    if frontier[i][0] > priority:
                        frontier.insert(i, (priority, next_tuple, next_state))
                        inserted = True
                        break
                if not inserted:
                    frontier.append((priority, next_tuple, next_state))
                
                came_from[next_tuple] = current_tuple

    return None

### BFS approach


In [8]:
def bfs_solve(start_state: np.ndarray, goal_state: np.ndarray) -> list[np.ndarray]:
    """Solve the n^2-1 puzzle using the BFS algorithm"""
    queue = [(start_state, [])]
    visited = set()

    while queue:
        current_state, path = queue.pop(0)

        if is_goal(current_state, goal_state):
            return path + [current_state]

        current_tuple = tuple(current_state.flatten())
        if current_tuple in visited:
            continue
        visited.add(current_tuple)

        for act in available_actions(current_state):
            next_state = do_action(current_state, act)
            next_tuple = tuple(next_state.flatten())

            if next_tuple not in visited:
                queue.append((next_state, path + [current_state]))

    return None

### IDDFS approach

In [9]:
def iddfs_solve(start_state: np.ndarray, goal_state: np.ndarray, max_depth=30) -> list[np.ndarray]:
    """Solve the n^2-1 puzzle using Iterative Deepening DFS (IDDFS)."""
    start_tuple = tuple(start_state.flatten())
    goal_tuple = tuple(goal_state.flatten())

    def dfs(current_tuple, path, depth, visited):
        if depth == 0:
            return None
        if current_tuple == goal_tuple:
            return path + [current_tuple]
        visited.add(current_tuple)
        current_state = np.array(current_tuple).reshape(start_state.shape)
        for act in available_actions(current_state):
            next_state = do_action(current_state, act)
            next_tuple = tuple(next_state.flatten())
            if next_tuple not in visited:
                result = dfs(next_tuple, path + [current_tuple], depth - 1, visited)
                if result:
                    return result
        visited.remove(current_tuple)
        return None

    for depth in range(1, max_depth + 1):
        visited = set()
        result = dfs(start_tuple, [], depth, visited)
        if result:
            return [np.array(p).reshape(start_state.shape) for p in result]

    return None

In [10]:
RANDOMIZE_STEPS = 100_000
state = np.array([i for i in range(1, PUZZLE_DIM**2)] + [0]).reshape((PUZZLE_DIM, PUZZLE_DIM))
for r in tqdm(range(RANDOMIZE_STEPS), desc='Randomizing'):
    state = do_action(state, choice(available_actions(state)))
state

Randomizing:   0%|          | 0/100000 [00:00<?, ?it/s]

array([[2, 3, 4],
       [5, 0, 6],
       [8, 7, 1]])

In [11]:
goal_state = np.array([i for i in range(1, PUZZLE_DIM**2)] + [0]).reshape((PUZZLE_DIM, PUZZLE_DIM))
goal_state

array([[1, 2, 3],
       [4, 5, 6],
       [7, 8, 0]])

In [12]:
solution = a_star_solve(state, goal_state)
if solution:
    print(f"Solution found in {len(solution) - 1} moves:")
    print(f"Step 0:\n{solution[0]}\n")
    print(f"Final Step ({len(solution) - 1}):\n{solution[-1]}\n")
else:
    print("No solution found.")

Solution found in 22 moves:
Step 0:
[[2 3 4]
 [5 0 6]
 [8 7 1]]

Final Step (22):
[[1 2 3]
 [4 5 6]
 [7 8 0]]



In [13]:
solution = bfs_solve(state, goal_state)
if solution:
    print(f"Solution found in {len(solution) - 1} moves:")
    print(f"Step 0:\n{solution[0]}\n")
    print(f"Final Step ({len(solution) - 1}):\n{solution[-1]}\n")
else:
    print("No solution found.")

Solution found in 22 moves:
Step 0:
[[2 3 4]
 [5 0 6]
 [8 7 1]]

Final Step (22):
[[1 2 3]
 [4 5 6]
 [7 8 0]]



In [14]:
solution = iddfs_solve(state, goal_state)
if solution:
    print(f"Solution found in {len(solution) - 1} moves:")
    print(f"Step 0:\n{solution[0]}\n")
    print(f"Final Step ({len(solution) - 1}):\n{solution[-1]}\n")
else:
    print("No solution found.")

Solution found in 22 moves:
Step 0:
[[2 3 4]
 [5 0 6]
 [8 7 1]]

Final Step (22):
[[1 2 3]
 [4 5 6]
 [7 8 0]]

